In [144]:
from io import BytesIO
import pandas as pd
import requests
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go 
#dash.__version__

## Получаем таблицу

In [145]:
from io import BytesIO
r = requests.get('https://docs.google.com/spreadsheets/d/1x85oldnFJr2SqHQhvhTVYj08T62FbIiwL9ub2QB9TZY/export?format=csv')
data = r.content
df = pd.read_csv(BytesIO(data), index_col=0).reset_index()
df.columns = ['timestamp','gender','age','city','most_difficult_theme','quality_rate','job_rate','review']
df['timestamp']= pd.to_datetime(df['timestamp'], format='%d.%m.%Y %H:%M:%S')
df['day'] = df['timestamp'].astype('datetime64[D]')
#df.timestamp = df.timestamp.astype('datetime64[h]')

In [146]:
df

,timestamp,gender,age,city,most_difficult_theme,quality_rate,job_rate,review,day
0,2020-02-04 18:13:05,М,22,Санкт-Петербург,Статистический анализ данных,6,6,NaN,2020-02-04
1,2020-02-04 21:49:47,М,2,Москва,Сбор и хранение данных,4,4,NaN,2020-02-04
2,2020-02-04 21:50:34,М,22,Москва,Анализ бизнес-показателей,4,7,NaN,2020-02-04
3,2020-02-04 22:11:59,Ж,2,Москва,Анализ бизнес-показателей,3,1,NaN,2020-02-04
4,2020-02-05 02:13:47,Ж,22,Москва,Принятие решений в бизнесе на основе данных,7,5,NaN,2020-02-05
5,2020-02-05 02:33:44,М,28,Санкт-Петербург,Анализ бизнес-показателей,7,5,NaN,2020-02-05
6,2020-02-05 14:01:13,М,5,Москва,Анализ бизнес-показателей,10,2,3.0,2020-02-05
7,2020-02-05 14:02:49,Ж,27,Ростов-на-Дону,Статистический анализ данных,9,10,7.0,2020-02-05
8,2020-02-05 14:12:31,М,29,Москва,Сложностей не возникло,8,3,7.0,2020-02-05
9,2020-02-05 22:16:47,Ж,26,Москва,Анализ бизнес-показателей,7,8,4.0,2020-02-05


## форматируем столбцы

In [147]:
df2 = df.groupby('gender').agg({'timestamp':'count','age':'mean',
                                'quality_rate':'mean','job_rate':'mean'}).reset_index()

'

In [148]:
df

,timestamp,gender,age,city,most_difficult_theme,quality_rate,job_rate,review,day
0,2020-02-04 18:13:05,М,22,Санкт-Петербург,Статистический анализ данных,6,6,NaN,2020-02-04
1,2020-02-04 21:49:47,М,2,Москва,Сбор и хранение данных,4,4,NaN,2020-02-04
2,2020-02-04 21:50:34,М,22,Москва,Анализ бизнес-показателей,4,7,NaN,2020-02-04
3,2020-02-04 22:11:59,Ж,2,Москва,Анализ бизнес-показателей,3,1,NaN,2020-02-04
4,2020-02-05 02:13:47,Ж,22,Москва,Принятие решений в бизнесе на основе данных,7,5,NaN,2020-02-05
5,2020-02-05 02:33:44,М,28,Санкт-Петербург,Анализ бизнес-показателей,7,5,NaN,2020-02-05
6,2020-02-05 14:01:13,М,5,Москва,Анализ бизнес-показателей,10,2,3.0,2020-02-05
7,2020-02-05 14:02:49,Ж,27,Ростов-на-Дону,Статистический анализ данных,9,10,7.0,2020-02-05
8,2020-02-05 14:12:31,М,29,Москва,Сложностей не возникло,8,3,7.0,2020-02-05
9,2020-02-05 22:16:47,Ж,26,Москва,Анализ бизнес-показателей,7,8,4.0,2020-02-05


In [149]:
#df2.iloc[:,2:5].apply(lambda x: x.round(2),axis=1)

Каждый раз когда скрипт обновляется в таблице становится больше данных

In [150]:
# создадим таблицу со сложными темами и кол-вом
most_dif = df.groupby('most_difficult_theme')['most_difficult_theme'].count()
most_dif.name = 'count'
most_dif = most_dif.reset_index()

In [151]:
most_dif 

,most_difficult_theme,count
0,Анализ бизнес-показателей,5
1,Принятие решений в бизнесе на основе данных,1
2,Сбор и хранение данных,1
3,Сложностей не возникло,1
4,Статистический анализ данных,2


In [152]:
x = most_dif['most_difficult_theme']
y = most_dif['count'].sort_values(ascending = False)
colors = ['lightslategray',] * 5
colors[0] = 'crimson'

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
            marker_color=colors
        )])

fig.show()

# Расчет рейтинга ЯП

In [153]:
# Временная замена на среднее 
df['review'].fillna(df.review.mean(),inplace=True)

In [154]:
# Столбец с рейтингом для каждого пользователя
df['pr_rate']= df.quality_rate*df.job_rate*df.review

In [155]:
df

,timestamp,gender,age,city,most_difficult_theme,quality_rate,job_rate,review,day,pr_rate
0,2020-02-04 18:13:05,М,22,Санкт-Петербург,Статистический анализ данных,6,6,5.25,2020-02-04,189.00
1,2020-02-04 21:49:47,М,2,Москва,Сбор и хранение данных,4,4,5.25,2020-02-04,84.00
2,2020-02-04 21:50:34,М,22,Москва,Анализ бизнес-показателей,4,7,5.25,2020-02-04,147.00
3,2020-02-04 22:11:59,Ж,2,Москва,Анализ бизнес-показателей,3,1,5.25,2020-02-04,15.75
4,2020-02-05 02:13:47,Ж,22,Москва,Принятие решений в бизнесе на основе данных,7,5,5.25,2020-02-05,183.75
5,2020-02-05 02:33:44,М,28,Санкт-Петербург,Анализ бизнес-показателей,7,5,5.25,2020-02-05,183.75
6,2020-02-05 14:01:13,М,5,Москва,Анализ бизнес-показателей,10,2,3.00,2020-02-05,60.00
7,2020-02-05 14:02:49,Ж,27,Ростов-на-Дону,Статистический анализ данных,9,10,7.00,2020-02-05,630.00
8,2020-02-05 14:12:31,М,29,Москва,Сложностей не возникло,8,3,7.00,2020-02-05,168.00
9,2020-02-05 22:16:47,Ж,26,Москва,Анализ бизнес-показателей,7,8,4.00,2020-02-05,224.00


In [156]:
# Средний рейтинг за все время на данный момент
pr_rate = (df.quality_rate*df.job_rate*df.review).mean()
pr_rate

188.525

In [157]:
# Таблица со средними рейтингами по дням
pr_rate_per_day = df.groupby('day')['pr_rate'].mean().round(2).reset_index()

In [158]:
pr_rate_per_day

,day,pr_rate
0,2020-02-04,108.94
1,2020-02-05,241.58


In [159]:
actual_delta = (pr_rate_per_day.iloc[-1,1] - pr_rate_per_day.iloc[-2,1]).round(2)

In [160]:
pr_rate_per_day.iloc[-1,1]

241.58

In [161]:
actual_delta

132.64

In [162]:
pr_day_rate = pr_rate_per_day.iloc[-2,1]

In [163]:
last_day_rate = pr_rate_per_day.iloc[-1,1]

### График по средним значениям

In [164]:
fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = last_day_rate,
    delta = {"reference": pr_day_rate, "valueformat": ".0f"},
    title = {"text": "Praktikum rate",
             "align" : 'center',
             "font": {
        "family": "Arial",
        "color": "red",
        
      },
             
             
            },
    
    domain = {'y': [0, 1], 'x': [0.25, 0.75]}))

fig.add_trace(go.Scatter(
    x = pr_rate_per_day['day'],
    y = pr_rate_per_day['pr_rate']))

fig.update_layout()
fig.show()

In [165]:
df

,timestamp,gender,age,city,most_difficult_theme,quality_rate,job_rate,review,day,pr_rate
0,2020-02-04 18:13:05,М,22,Санкт-Петербург,Статистический анализ данных,6,6,5.25,2020-02-04,189.00
1,2020-02-04 21:49:47,М,2,Москва,Сбор и хранение данных,4,4,5.25,2020-02-04,84.00
2,2020-02-04 21:50:34,М,22,Москва,Анализ бизнес-показателей,4,7,5.25,2020-02-04,147.00
3,2020-02-04 22:11:59,Ж,2,Москва,Анализ бизнес-показателей,3,1,5.25,2020-02-04,15.75
4,2020-02-05 02:13:47,Ж,22,Москва,Принятие решений в бизнесе на основе данных,7,5,5.25,2020-02-05,183.75
5,2020-02-05 02:33:44,М,28,Санкт-Петербург,Анализ бизнес-показателей,7,5,5.25,2020-02-05,183.75
6,2020-02-05 14:01:13,М,5,Москва,Анализ бизнес-показателей,10,2,3.00,2020-02-05,60.00
7,2020-02-05 14:02:49,Ж,27,Ростов-на-Дону,Статистический анализ данных,9,10,7.00,2020-02-05,630.00
8,2020-02-05 14:12:31,М,29,Москва,Сложностей не возникло,8,3,7.00,2020-02-05,168.00
9,2020-02-05 22:16:47,Ж,26,Москва,Анализ бизнес-показателей,7,8,4.00,2020-02-05,224.00


## Надо сделать интерактивный график с возможностью выбора трэйс(qual,job,review)

In [166]:
raters = df.groupby('day')[['quality_rate','job_rate','review']].mean().reset_index()

In [167]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = raters['day'],
    y = raters['quality_rate']))

fig.add_trace(go.Scatter(
    x = raters['day'],
    y = raters['job_rate']))

fig.add_trace(go.Scatter(
    x = raters['day'],
    y = raters['review']))

fig.update_layout()
fig.show()

In [168]:
dcc.Dropdown(
    options=[
        {'label': 'New York City', 'value': 'NYC'},
        {'label': 'Montréal', 'value': 'MTL'},
        {'label': 'San Francisco', 'value': 'SF'}
    ],
    value='MTL'
)  

Dropdown(options=[{'label': 'New York City', 'value': 'NYC'}, {'label': 'Montréal', 'value': 'MTL'}, {'label': 'San Francisco', 'value': 'SF'}], value='MTL')

In [169]:
df

,timestamp,gender,age,city,most_difficult_theme,quality_rate,job_rate,review,day,pr_rate
0,2020-02-04 18:13:05,М,22,Санкт-Петербург,Статистический анализ данных,6,6,5.25,2020-02-04,189.00
1,2020-02-04 21:49:47,М,2,Москва,Сбор и хранение данных,4,4,5.25,2020-02-04,84.00
2,2020-02-04 21:50:34,М,22,Москва,Анализ бизнес-показателей,4,7,5.25,2020-02-04,147.00
3,2020-02-04 22:11:59,Ж,2,Москва,Анализ бизнес-показателей,3,1,5.25,2020-02-04,15.75
4,2020-02-05 02:13:47,Ж,22,Москва,Принятие решений в бизнесе на основе данных,7,5,5.25,2020-02-05,183.75
5,2020-02-05 02:33:44,М,28,Санкт-Петербург,Анализ бизнес-показателей,7,5,5.25,2020-02-05,183.75
6,2020-02-05 14:01:13,М,5,Москва,Анализ бизнес-показателей,10,2,3.00,2020-02-05,60.00
7,2020-02-05 14:02:49,Ж,27,Ростов-на-Дону,Статистический анализ данных,9,10,7.00,2020-02-05,630.00
8,2020-02-05 14:12:31,М,29,Москва,Сложностей не возникло,8,3,7.00,2020-02-05,168.00
9,2020-02-05 22:16:47,Ж,26,Москва,Анализ бизнес-показателей,7,8,4.00,2020-02-05,224.00


In [170]:
def random_date_generator(start_date, range_in_days):
    days_to_add = np.arange(0, range_in_days)
    random_date = np.datetime64(start_date) + np.random.choice(days_to_add)
    return random_date

In [171]:
exampledatetime2 = np.datetime64('2017-01-01')


In [172]:
exampledatetime2 + 1


numpy.datetime64('2017-01-02')

In [173]:
bimonthly_days = np.arange(0, 60)
base_date = np.datetime64('2020-02-06')
random_date = base_date + np.random.choice(bimonthly_days)

In [174]:
base_date + np.random.choice(bimonthly_days)

numpy.datetime64('2020-03-18')

In [175]:
df2 = pd.DataFrame([['2020-02-05 22:16:47','Ж','23','Москва','Статистический анализ данных','7','8','9','2020-02-05','33']], 
                   columns=['timestamp','gender','age','city','most_difficult_theme','quality_rate','job_rate','review','day','pr_rate'])

df.append(df2, ignore_index=True)

,timestamp,gender,age,city,most_difficult_theme,quality_rate,job_rate,review,day,pr_rate
0,2020-02-04 18:13:05,М,22,Санкт-Петербург,Статистический анализ данных,6,6,5.25,2020-02-04 00:00:00,189
1,2020-02-04 21:49:47,М,2,Москва,Сбор и хранение данных,4,4,5.25,2020-02-04 00:00:00,84
2,2020-02-04 21:50:34,М,22,Москва,Анализ бизнес-показателей,4,7,5.25,2020-02-04 00:00:00,147
3,2020-02-04 22:11:59,Ж,2,Москва,Анализ бизнес-показателей,3,1,5.25,2020-02-04 00:00:00,15.75
4,2020-02-05 02:13:47,Ж,22,Москва,Принятие решений в бизнесе на основе данных,7,5,5.25,2020-02-05 00:00:00,183.75
5,2020-02-05 02:33:44,М,28,Санкт-Петербург,Анализ бизнес-показателей,7,5,5.25,2020-02-05 00:00:00,183.75
6,2020-02-05 14:01:13,М,5,Москва,Анализ бизнес-показателей,10,2,3,2020-02-05 00:00:00,60
7,2020-02-05 14:02:49,Ж,27,Ростов-на-Дону,Статистический анализ данных,9,10,7,2020-02-05 00:00:00,630
8,2020-02-05 14:12:31,М,29,Москва,Сложностей не возникло,8,3,7,2020-02-05 00:00:00,168
9,2020-02-05 22:16:47,Ж,26,Москва,Анализ бизнес-показателей,7,8,4,2020-02-05 00:00:00,224
